In [1]:
import pymysql
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import pyqtSlot
import matplotlib.pyplot as plt
# pyqt5를 지원하는 matplotlib 모듈 호출
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas

import pandas as pd
from pandas import Series, DataFrame

In [2]:
user_ip =""
user_id=""
user_password=""
node_number = ""

In [3]:
class Models():
    def get_Model(self):
        global user_ip
        global user_id
        global user_password
        return user_ip,user_id,user_password
    def set_Model(self,ip,ids,password):
        global user_ip 
        global user_id 
        global user_password 
        user_ip = ip
        user_id = ids
        user_password = password

In [4]:
class Node_Model:
    def get_Node(self):
        global node_number
        return node_number
    
    def set_Node(self,node):
        global node_number
        node_number = node

In [5]:
class LogInDialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setupUI()
        
        self.ip = None
        self.id = None
        self.password = None

    def setupUI(self):
        self.setGeometry(1100, 200, 300, 100)
        self.setWindowTitle("Sign In")
        #self.setWindowIcon(QIcon('icon.png'))
    
        label1 = QLabel("IP: ")
        label2 = QLabel("USER: ")
        label3 = QLabel("Password: ")

        self.lineEdit1 = QLineEdit() # ip 입력
        self.lineEdit2 = QLineEdit() # user이름 입력
        self.lineEdit3 = QLineEdit() # 비밀번호 입력
        self.pushButton1= QPushButton("Connect")
        self.pushButton1.clicked.connect(self.pushButtonClicked)

        layout = QGridLayout()
        layout.addWidget(label1, 0, 0)
        layout.addWidget(self.lineEdit1, 0, 1)
        layout.addWidget(self.pushButton1, 0, 2)
        layout.addWidget(label2, 1, 0)
        layout.addWidget(self.lineEdit2, 1, 1)
        layout.addWidget(label3, 2, 0)
        layout.addWidget(self.lineEdit3, 2, 1)

        self.setLayout(layout)
        
    def pushButtonClicked(self):
        
        if (not self.lineEdit1.text()):
            QMessageBox.about(self, "경고", "정확한 주소를 입력해주세요")
        elif(not self.lineEdit2.text()):
            QMessageBox.about(self, "경고", "정확한 아이디를 입력해주세요")
        else:
            self.ip = self.lineEdit1.text()
            self.id = self.lineEdit2.text()
            self.password = self.lineEdit3.text()
        self.close()

In [6]:
class HelperDialog(QDialog):

    def __init__(self):
        super().__init__()
        self.setupUI()
        
        test = Models()
        node_Model = Node_Model()
        
        self.ip, self.id , self.password = test.get_Model()
        self.node_num = node_Model.get_Node()
        self.chart_flag = True
        
        #self.ip = global user_ip
        #self.id = global user_id
        #self.password = global user_password

 
    def setupUI(self):
        self.setGeometry(1100, 200, 600, 600)
        self.setWindowTitle("노드 온도 습도 그래프")
        #self.setWindowIcon(QIcon('icon.png'))
        
        self.fig = plt.Figure()
        self.canvas = FigureCanvas(self.fig)
    
#         self.label1 = QLabel("IP: ")
#         self.label2 = QLabel("USER: ")
#         self.label3 = QLabel("Password: ")
        self.label4 = QLabel()
    
        # 모든 데이터를 차트에 표기하는 테이블 객체를 생성
        self.tableWidget = QTableWidget(self)
        self.tableWidget.setRowCount(200) # seq, node , temp, humi, date, time
        self.tableWidget.setColumnCount(6)
        self.tableWidget.setEditTriggers(QAbstractItemView.NoEditTriggers)
        
#         self.lineEdit1 = QLineEdit() # ip 입력
#         self.lineEdit2 = QLineEdit() # user이름 입력
#         self.lineEdit3 = QLineEdit() # 비밀번호 입력
        self.pushButton1= QPushButton("보기")
        self.pushButton1.clicked.connect(self.pushButtonClicked)
        
        self.pushButton2= QPushButton("닫기")
        self.pushButton2.clicked.connect(self.pushButtonClicked2)

        layout = QVBoxLayout()
        layout.addWidget(self.label4)
        layout.addWidget(self.pushButton1)
        layout.addWidget(self.canvas)
        layout.addWidget(self.tableWidget)
        layout.addWidget(self.pushButton2)
        
#         layout.addWidget(self.lineEdit1, 0, 1)
#         layout.addWidget(self.pushButton1, 0, 2)
#         layout.addWidget(self.label2, 1, 0)
#         layout.addWidget(self.lineEdit2, 1, 1)
#         layout.addWidget(self.label3, 2, 0)
#         layout.addWidget(self.lineEdit3, 2, 1)
#         layout.addWidget(self.label4, 3, 0)
        
          #host_ip = self.lineEdit.text()
        #self.host_ip = self.ip
        self.setLayout(layout)
        
    def setTableNodeData(self, node_num):
                
#         column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4}
#         column_headers = ['seq', 'node', 'temp', 'humi', 'date']
        
        column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4, 'time':5}
        column_headers = ['seq', 'node', 'temp', 'humi', 'date', 'time']
        self.tableWidget.setHorizontalHeaderLabels(column_headers)
        
        seq_list = []
        node_list = []
        temp_list = []
        humi_list = []
        date_list = []
        time_list = []
        db_dict = {}
        
        if (not self.ip) and (not self.id) and (not self.password):
            self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
            QMessageBox.about(self, "경고", "데이터베이스를 연결해주세요.")
        else: 
            conn = pymysql.connect(host= self.ip, user=self.id, password=self.password, db='usn', charset='utf8')
            curs = conn.cursor(pymysql.cursors.DictCursor)
            sql = """select * from usnnode where node=""" + node_num
            curs.execute(sql)
            rows = curs.fetchall()
            
            db_dict.clear()
            seq_list.clear()
            node_list.clear()
            temp_list.clear()
            humi_list.clear()
            date_list.clear()
            time_list.clear()
            
            length = len(rows)
            for i in range(length):
                seq = rows[i]['seq']
                node = rows[i]['node']
                temp = rows[i]['temp']
                humi = rows[i]['humi']
                datetime = rows[i]['date'].split()
                db_date = datetime[0]
                db_time = datetime[1]
                #db_date = rows[i]['date']
                #db_time = rows[i]['time']
                seq_list.append(seq)
                node_list.append(node)
                temp_list.append(temp)
                humi_list.append(humi)
                date_list.append(db_date)
                time_list.append(db_time)

            db_dict['seq'] = seq_list
            db_dict['node'] = node_list
            db_dict['temp'] = temp_list
            db_dict['humi'] = humi_list
            db_dict['date'] = date_list
            db_dict['time'] = time_list
            
            for k, v in db_dict.items():
                col = column_idx_lookup[k]
                for row, val in enumerate(v):
                    item = QTableWidgetItem(val)
                    self.tableWidget.setItem(row, col, item)
                
        self.tableWidget.resizeColumnsToContents()
        self.tableWidget.resizeRowsToContents()
        
    def pushButtonClicked(self):
        
        self.label4.setText("node num = : %s, type : %s, %s, node_num = %s, type %s" % (self.ip, self.id, self.password, self.node_num,\
                                                                                        type(self.node_num)))
        
#         if (not self.lineEdit1.text()):
#             QMessageBox.about(self, "경고", "정확한 주소를 입력해주세요")
#         elif(not self.lineEdit2.text()):
#             QMessageBox.about(self, "경고", "정확한 아이디를 입력해주세요")
#         else:
#             self.ip = self.lineEdit1.text()
#             self.id = self.lineEdit2.text()
#             self.password = self.lineEdit3.text()
#         #self.close()
        
        if (not self.ip) and (not self.id) and (not self.password):
            self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
            QMessageBox.about(self, "경고", "데이터베이스를 연결해주세요.")
        else:
            #conn = pymysql.connect(host= self.host_ip, user='root', password='1234', db='usn', charset='utf8')
            conn = pymysql.connect(host= self.ip, user=self.id, password=self.password, db='usn', charset='utf8')
            sql = """select * from usnnode where node=""" + self.node_num
            df = pd.read_sql(sql, con = conn)
#             df['T_MA20'] = df['temp'].rolling(window=20).mean()
#             df['T_MA60'] = df['temp'].rolling(window=60).mean()
#             df['H_MA20'] = df['humi'].rolling(window=20).mean()
#             df['H_MA60'] = df['humi'].rolling(window=60).mean() # 이동 평균 윈도우 : 60
        
        if self.chart_flag:
            temp_plot = self.fig.add_subplot(111)
            temp_plot.plot(df.index, df['temp'],label='temp')
            #humi_plot = self.fig_2.add_subplot(212)
            temp_plot.plot(df.index, df['humi'],label='humi')
#             temp_plot.plot(df.index, df['T_MA20'],label='tmp_mvmean20')
#             temp_plot.plot(df.index, df['T_MA60'],label='tmp_mvmean60')
#             temp_plot.plot(df.index, df['H_MA20'],label='humi_mvmean20')
#             temp_plot.plot(df.index, df['H_MA60'],label='humi_mvmean60')
            temp_plot.legend(loc='best')
            #temp_plot.legend(loc='upper right')
            temp_plot.grid()
            #humi_plot.grid()
            self.canvas.draw()
            #self.canvas_2.draw()
            #self.setTableWidgetData()
            self.setTableNodeData(self.node_num)
            self.chart_flag = False
        else:
            pass
        
       
        
    def pushButtonClicked2(self):
        self.close()
    
   

In [7]:
class MyWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.setupUI()
        
        self.ip = None
        self.id = None
        self.password = None
        self.host_ip = None
        self.node_val = None
        
        self.chart_flag = True

    def setupUI(self):
        self.setGeometry(600, 200, 1200, 600)
        self.setWindowTitle("Usn Chart Viewer v0.1")
        #self.setWindowIcon(QIcon('icon.png'))
        
        #메뉴바 만들려다가 왜 그런지 잘 안됌...
        
#         self.mainMenu = QMenuBar()
#         self.fileMenu = self.mainMenu.addMenu('File')
#         self.editMenu = self.mainMenu.addMenu('Edit')
#         self.viewMenu = self.mainMenu.addMenu('View')
#         self.searchMenu = self.mainMenu.addMenu('Search')
#         self.toolsMenu = self.mainMenu.addMenu('Tools')
#         self.helpMenu = self.mainMenu.addMenu('Help')
#         self.exitButton = QAction('Exit', self)
#         self.exitButton.setShortcut('Ctrl+Q')
#         self.exitButton.setStatusTip('Exit application')
#         self.exitButton.triggered.connect(self.close)
#         self.fileMenu.addAction(self.exitButton)
        
        label = QLabel() # 디비 아이피 정보 및 유저 정보 
        
        groupBox = QGroupBox("데이터베이스 연결", self)
        groupBox2 = QGroupBox("데이터 확인", self)
        groupBox3 = QGroupBox("개별 노드 데이터 확인", self)
        groupBox4 = QGroupBox("센서 값 그래프", self)
        #groupBox.resize(280, 80)
        
        self.label = QLabel()
        self.lineEdit = QLineEdit()
        self.pushButton1 = QPushButton("데이터베이스 접속")
        self.pushButton1.clicked.connect(self.pushDBButtonClicked)  
        self.pushButton2 = QPushButton("데이터 보기")
        self.pushButton2.clicked.connect(self.pushButtonClicked1)   
        self.pushButton3 = QPushButton("차트 그리기")
        self.pushButton3.clicked.connect(self.pushButtonClicked2)   
        
        self.pushButton4 = QPushButton("데이터 확인")
        self.pushButton4.clicked.connect(self.pushButtonClicked3)   
        
        self.pushButton5 = QPushButton("노드 그래프 보기")
        self.pushButton5.clicked.connect(self.pushButtonClicked4)   
        
        # 모든 데이터를 차트에 표기하는 테이블 객체를 생성
        self.tableWidget = QTableWidget(self)
        self.tableWidget.setRowCount(200) # seq, node , temp, humi, date, time
        self.tableWidget.setColumnCount(6)
        self.tableWidget.setEditTriggers(QAbstractItemView.NoEditTriggers)
        
        #노드를 선택하기위한 spinbox 객체 생성.
        self.node_label = QLabel("노드 번호 입력: ", self)
        self.spinBox = QSpinBox(self)
        self.spinBox.valueChanged.connect(self.spinBoxChanged)
        
        # 특정 노드의 데이터를 차트에 표기하는 테이블 객체를 생성
        self.tableWidget2 = QTableWidget(self)
        self.tableWidget2.setRowCount(200) # seq, node , temp, humi, date, time
        self.tableWidget2.setColumnCount(6)
        self.tableWidget2.setEditTriggers(QAbstractItemView.NoEditTriggers)
        
        # FigureCanvas 객체를 생성한다.
        self.fig = plt.Figure()
        self.canvas = FigureCanvas(self.fig)
        
#         self.fig_2 = plt.Figure()
#         self.canvas_2 = FigureCanvas(self.fig_2)

        # 왼쪽의 뷰를 정의한다.
        LeftInnerLayOut = QVBoxLayout()
        LeftInnerLayOut.addWidget(self.canvas)
        groupBox4.setLayout(LeftInnerLayOut)
        leftLayout = QVBoxLayout()
        leftLayout.addWidget(groupBox4)
#         leftLayout.addWidget(self.canvas_2)
        # 오른쪽의 뷰를 정의한다.
        RightInnerLayOut = QVBoxLayout()
        RightInnerLayOut.addWidget(self.lineEdit)
        RightInnerLayOut.addWidget(self.pushButton1)
        groupBox.setLayout(RightInnerLayOut)
        
        RightInnerLayOut2 = QVBoxLayout()
        RightInnerLayOut2.addWidget(self.pushButton2)
        RightInnerLayOut2.addWidget(self.pushButton3)
        RightInnerLayOut2.addWidget(self.tableWidget)
        groupBox2.setLayout(RightInnerLayOut2)
        
        RightInnerLayOut3_1 = QHBoxLayout()
        RightInnerLayOut3_1.addWidget(self.node_label)
        RightInnerLayOut3_1.addWidget(self.spinBox)
        RightInnerLayOut3_1.addWidget(self.pushButton4)
        RightInnerLayOut3 = QVBoxLayout()
        RightInnerLayOut3.addLayout(RightInnerLayOut3_1)
        RightInnerLayOut3.addWidget(self.tableWidget2)
        RightInnerLayOut3.addWidget(self.pushButton5)
        groupBox3.setLayout(RightInnerLayOut3)
        
        # Right Layout
        rightLayout = QVBoxLayout()
        rightLayout.addWidget(groupBox)
        rightLayout.addWidget(groupBox2)
        rightLayout.addWidget(self.label)
        rightLayout.addWidget(groupBox3)
       
        rightLayout.addStretch(1) # 크기 조절이 가능한 공백을 추가

        layout = QHBoxLayout()
        layout.addLayout(leftLayout)
        layout.addLayout(rightLayout)
        layout.setStretchFactor(leftLayout, 1)
        layout.setStretchFactor(rightLayout, 0)

        self.setLayout(layout)
        
    def spinBoxChanged(self):
        node_class = Node_Model()
        
        self.node_val = self.spinBox.value()
        self.node_val = str(self.node_val)
        self.label.setText("node num = : %s, type : %s" % (self.node_val, type(self.node_val)))
        node_class.set_Node(self.node_val)
        
        #msg = '%d 주를 매도합니다.' % (val)
        #self.statusBar.showMessage(msg)
        
    #테이블의 데이터를 가져오는 함수.
    def pushButtonClicked1(self):   
        self.setTableWidgetData()
    #개별 노드의 테이블의 데이터를 가져오는 함수.
    def pushButtonClicked3(self):   
        self.setTableEachNodeData(self.node_val)
    #차트를 그리는 함수..    
    def pushButtonClicked2(self): 
        
        #host_ip = self.lineEdit.text()
        #self.host_ip = self.ip
        if (not self.ip) and (not self.id) and (not self.password):
            self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
            QMessageBox.about(self, "경고", "데이터베이스를 연결해주세요.")
        else:
            #conn = pymysql.connect(host= self.host_ip, user='root', password='1234', db='usn', charset='utf8')
            conn = pymysql.connect(host= self.ip, user=self.id, password=self.password, db='usn', charset='utf8')
            df = pd.read_sql('select * from usnnode', con = conn)
            df['T_MA20'] = df['temp'].rolling(window=20).mean()
            df['T_MA60'] = df['temp'].rolling(window=60).mean()
            df['H_MA20'] = df['humi'].rolling(window=20).mean()
            df['H_MA60'] = df['humi'].rolling(window=60).mean() # 이동 평균 윈도우 : 60
        
        if self.chart_flag:
            temp_plot = self.fig.add_subplot(111)
            temp_plot.plot(df.index, df['temp'],label='temp')
            #humi_plot = self.fig_2.add_subplot(212)
            temp_plot.plot(df.index, df['humi'],label='humi')
            temp_plot.plot(df.index, df['T_MA20'],label='tmp_mvmean20')
            temp_plot.plot(df.index, df['T_MA60'],label='tmp_mvmean60')
            temp_plot.plot(df.index, df['H_MA20'],label='humi_mvmean20')
            temp_plot.plot(df.index, df['H_MA60'],label='humi_mvmean60')
            temp_plot.legend(loc='best')
            #temp_plot.legend(loc='upper right')
            temp_plot.grid()
            #humi_plot.grid()
            self.canvas.draw()
            #self.canvas_2.draw()
            #self.setTableWidgetData()
            self.chart_flag = False
        else:
            pass
        
    def pushDBButtonClicked(self):
         
        dlg = LogInDialog()
        dlg.exec_()
        
        self.ip = dlg.ip
        self.id = dlg.id
        self.password = dlg.password
        
        test_model = Models()
        test_model.set_Model(self.ip,self.id,self.password)
        
#         user_ip = dlg.ip
#         user_id = dlg.id
#         user_password = dlg.password
        print(user_ip, user_id, user_password)
        
        if (self.ip is None) and (self.id is None) and (self.password is None):
            self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
        else:
            self.label.setText("id: %s password: %s type %s, %s" % (self.ip, self.id, type(self.ip), type(self.id)))
#         host_ip = self.lineEdit.text()
#         conn = pymysql.connect(host= host_ip, user='root', password='1234', db='usn', charset='utf8')
#         df = pd.read_sql('select * from usnnode', con = conn)
#         temp_plot = self.fig.add_subplot(211)
#         temp_plot.plot(df.index, df['temp'],label='temp')
#         temp_plot.legend(loc='upper right')
#         humi_plot = self.fig_2.add_subplot(212)
#         humi_plot.plot(df.index, df['humi'],label='humi')
#         humi_plot.legend(loc='upper right')
#         temp_plot.grid()
#         humi_plot.grid()
        
#         self.canvas.draw()
#         self.canvas_2.draw()
        
#         self.setTableWidgetData()

    def pushButtonClicked4(self):
         
        dlg = HelperDialog()
        dlg.exec_()
        
#         self.ip = dlg.ip
#         self.id = dlg.id
#         self.password = dlg.password
        
#         if (self.ip is None) and (self.id is None) and (self.password is None):
#             self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
#         else:
        self.label.setText("id: %s password: %s type %s, %s" % (self.ip, self.id, type(self.ip), type(self.id)))
#         host_ip = self.lineEdit.text()
#         conn = pymysql.connect(host= host_ip, user='root', password='1234', db='usn', charset='utf8')
#         df = pd.read_sql('select * from usnnode', con = conn)
#         temp_plot = self.fig.add_subplot(211)
#         temp_plot.plot(df.index, df['temp'],label='temp')
#         temp_plot.legend(loc='upper right')
#         humi_plot = self.fig_2.add_subplot(212)
#         humi_plot.plot(df.index, df['humi'],label='humi')
#         humi_plot.legend(loc='upper right')
#         temp_plot.grid()
#         humi_plot.grid()
        
#         self.canvas.draw()
#         self.canvas_2.draw()
        
#         self.setTableWidgetData()
        
    def setTableWidgetData(self):
                
#         column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4}
#         column_headers = ['seq', 'node', 'temp', 'humi', 'date']
        
        column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4, 'time':5}
        column_headers = ['seq', 'node', 'temp', 'humi', 'date', 'time']
        self.tableWidget.setHorizontalHeaderLabels(column_headers)
        
        if (not self.ip) and (not self.id) and (not self.password):
            self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
            QMessageBox.about(self, "경고", "데이터베이스를 연결해주세요.")
        else: 
            conn = pymysql.connect(host= self.ip, user=self.id, password=self.password, db='usn', charset='utf8')
            
            curs = conn.cursor(pymysql.cursors.DictCursor)
            curs.execute("""select * from usnnode""")
            rows = curs.fetchall()
            seq_list = []
            node_list = []
            temp_list = []
            humi_list = []
            date_list = []
            time_list = []
            db_dict = {}
            length = len(rows)
            for i in range(length):
                seq = rows[i]['seq']
                node = rows[i]['node']
                temp = rows[i]['temp']
                humi = rows[i]['humi']
                datetime = rows[i]['date'].split()
                db_date = datetime[0]
                db_time = datetime[1]
                #db_date = rows[i]['date']
                #db_time = rows[i]['time']
                seq_list.append(seq)
                node_list.append(node)
                temp_list.append(temp)
                humi_list.append(humi)
                date_list.append(db_date)
                time_list.append(db_time)

            db_dict['seq'] = seq_list
            db_dict['node'] = node_list
            db_dict['temp'] = temp_list
            db_dict['humi'] = humi_list
            db_dict['date'] = date_list
            db_dict['time'] = time_list
            for k, v in db_dict.items():
                col = column_idx_lookup[k]
                for row, val in enumerate(v):
                    item = QTableWidgetItem(val)
                    self.tableWidget.setItem(row, col, item)
                
        self.tableWidget.resizeColumnsToContents()
        self.tableWidget.resizeRowsToContents()
        
    def setTableEachNodeData(self, node_num):
                
#         column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4}
#         column_headers = ['seq', 'node', 'temp', 'humi', 'date']
        
        column_idx_lookup = {'seq': 0, 'node': 1, 'temp': 2, 'humi' : 3, 'date' : 4, 'time':5}
        column_headers = ['seq', 'node', 'temp', 'humi', 'date', 'time']
        self.tableWidget2.setHorizontalHeaderLabels(column_headers)
        
        seq_list = []
        node_list = []
        temp_list = []
        humi_list = []
        date_list = []
        time_list = []
        db_dict = {}
        
        if (not self.ip) and (not self.id) and (not self.password):
            self.label.setText("접속할 데이터베이스의 정보를 입력해주세요")
            QMessageBox.about(self, "경고", "데이터베이스를 연결해주세요.")
        else: 
            conn = pymysql.connect(host= self.ip, user=self.id, password=self.password, db='usn', charset='utf8')
            curs = conn.cursor(pymysql.cursors.DictCursor)
            sql = """select * from usnnode where node=""" + node_num
            curs.execute(sql)
            rows = curs.fetchall()
            
            db_dict.clear()
            seq_list.clear()
            node_list.clear()
            temp_list.clear()
            humi_list.clear()
            date_list.clear()
            time_list.clear()
            
            length = len(rows)
            for i in range(length):
                seq = rows[i]['seq']
                node = rows[i]['node']
                temp = rows[i]['temp']
                humi = rows[i]['humi']
                datetime = rows[i]['date'].split()
                db_date = datetime[0]
                db_time = datetime[1]
                #db_date = rows[i]['date']
                #db_time = rows[i]['time']
                seq_list.append(seq)
                node_list.append(node)
                temp_list.append(temp)
                humi_list.append(humi)
                date_list.append(db_date)
                time_list.append(db_time)

            db_dict['seq'] = seq_list
            db_dict['node'] = node_list
            db_dict['temp'] = temp_list
            db_dict['humi'] = humi_list
            db_dict['date'] = date_list
            db_dict['time'] = time_list
            
            for k, v in db_dict.items():
                col = column_idx_lookup[k]
                for row, val in enumerate(v):
                    item = QTableWidgetItem(val)
                    self.tableWidget2.setItem(row, col, item)
                
        self.tableWidget2.resizeColumnsToContents()
        self.tableWidget2.resizeRowsToContents()

In [8]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MyWindow()
    window.show()
    app.exec_()

127.0.0.1 root 1234
